In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from utils import plot_candlestick

In [ ]:
LOG_DIR = '../.log/'

BACKTEST_LOG = f"{LOG_DIR}backtest.json"
MARKET_LOG = f"{LOG_DIR}market.json"
PORTFOLIO_LOG = f"{LOG_DIR}portfolio.json"
SIGNAL_LOG = f"{LOG_DIR}signal.json"
POSITION_LOG = f"{LOG_DIR}position.json"

In [ ]:
backtest_df = pd.read_json(BACKTEST_LOG)
backtest_df_flat = pd.json_normalize(backtest_df.to_dict(orient='records'))

market_df = pd.read_json(MARKET_LOG)
market_df_flat = pd.json_normalize(market_df.to_dict(orient='records'))

signal_df = pd.read_json(SIGNAL_LOG)
signal_df_flat = pd.json_normalize(signal_df.to_dict(orient='records'))

position_df = pd.read_json(POSITION_LOG)
position_df_flat = pd.json_normalize(position_df.to_dict(orient='records'))

portfolio_df = pd.read_json(PORTFOLIO_LOG)
portfolio_df_flat = pd.json_normalize(portfolio_df.to_dict(orient='records'))

In [ ]:
backtest_df_flat.info()

In [ ]:
backtest_df_flat.describe()

In [ ]:
market_df_flat.info()

In [ ]:
market_df_flat.describe()

In [ ]:
signal_df_flat.info()

In [ ]:
signal_df_flat.describe()

In [ ]:
position_df_flat.info()

In [ ]:
position_df_flat.describe()

In [ ]:
portfolio_df_flat.info()

In [ ]:
portfolio_df_flat.describe()

In [ ]:
symbol_dataframes = {}

rename_map = {
    'ohlcv.timestamp': 'timestamp',
    'ohlcv.open': 'open',
    'ohlcv.high': 'high',
    'ohlcv.low': 'low',
    'ohlcv.close': 'close',
    'ohlcv.volume': 'volume'
}

for symbol in market_df_flat['symbol.name'].unique():
    if symbol is np.nan:
        continue
    
    symbol_df = market_df_flat[market_df_flat['symbol.name'] == symbol]
    
    ohlcv_df = symbol_df[list(rename_map.keys())].copy()
    ohlcv_df = ohlcv_df.rename(columns=rename_map)
    ohlcv_df['symbol'] = symbol
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], unit='ms')
    
    symbol_dataframes[symbol] = ohlcv_df

In [ ]:
symbol_dataframes.keys()

In [ ]:
fig = plot_candlestick(symbol_dataframes)
fig.show()